# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up



In [ ]:
# Do all imports and installs here


import os
import sys
import boto3
import datetime
import pandas as pd

In [ ]:
!{sys.executable} -m pip install -U ray
os.environ['PYSPARK_SUBMIT_ARGS'] = "--packages=org.apache.hadoop:hadoop-aws:2.7.3 pyspark-shell"

In [ ]:
import configparser
config = configparser.ConfigParser()
config.read('dl.cfg')
# set AWS variables
os.environ['AWS_ACCESS_KEY_ID']    = config['AWS']['KEY']
os.environ['AWS_SECRET_ACCESS_KEY'] = config['AWS']['SECRET']


### Step 1: Scope the Project and Gather Data

#### Scope TODO
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data TODO
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [ ]:
# Read in the data here
!ls -ls ./dataset
path = './dataset/'

In [ ]:
print("__________________________________________________________________________")
# airport-codes_csv
df_airport = pd.read_csv(path+'airport-codes_csv.csv')
print(df_airport.head(2))
print("__________________________________________________________________________")
# immigration_data_sample
df_immigration = pd.read_csv(path+'immigration_data_sample.csv')
print(df_immigration.head(2))
print("__________________________________________________________________________")
# us-cities-demographics
df_demograph = pd.read_csv(path+'us-cities-demographics.csv', sep=";")
print(df_demograph.head(2))
print("__________________________________________________________________________")
# download from kaggle the GlobalLandTemperaturesByCity.csv
df_temperature = pd.read_csv(path+'GlobalLandTemperaturesByCity.csv', sep=",")
print(df_temperature.head(2))
print("__________________________________________________________________________")

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()
#df_spark =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

In [ ]:
df_sas=spark.read.parquet("./dataset/sas_data")
df_sas

In [ ]:
print("df_airport shape : {}".format(df_airport.shape))
print("df_immigration shape : {}".format(df_immigration.shape))
print("df_demograph shape : {}".format(df_demograph.shape))
print("df_temperature shape : {}".format(df_temperature.shape))
print("df_sas shape : {}, {}".format(df_sas.count(), len(df_sas.columns)))

In [ ]:
from time import time

In [ ]:
s3 = boto3.resource('s3', 
                    aws_access_key_id=KEY,
                    aws_secret_access_key=SECRET,
                    region_name="us-west-1")
bucket=s3.Bucket("gdelt-open-data")
i=0
for obj in s3.Bucket(name="gdelt-open-data").objects.all():
    if i > 30:
        break
    i+=1
    print(os.path.join(obj.bucket_name, obj.key))

In [ ]:
#lines = sc.textFile("s3://gdelt-open-data/events/2016*")

size = sum(1 for _ in bucket.objects.filter(Prefix="events/2016"))
print(size)

In [ ]:
n = 0
for obj in bucket.objects.filter(Prefix="events/2016"):
    if n > 5:
        break
    n+=1
    print(obj)

In [ ]:
for obj in bucket.objects.filter(Prefix='events/20160201.export.csv'):
    body_song = obj.get()['Body'].read().decode('utf-8').split()

In [ ]:
bucket="gdelt-open-data"
file_name="events/20160101.export.csv"
obj = s3.get_object(Bucket= bucket, Key= file_name) 
initial_df = pd.read_csv(obj['Body'])

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [ ]:
# Performing cleaning tasks here


### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.